In [1]:
import logging
logging.basicConfig(level=logging.INFO)
import numpy as np
import time
import awa_interface
from observations.yag_screen import YAGScreen
from xopt.bayesian.algorithms import bayesian_optimize
import torch

In [2]:


from botorch.acquisition.analytic import UpperConfidenceBound

def acq(model):
    return UpperConfidenceBound(model, beta=2.0)

class Evaluator:
    def __init__(self, interface):
        self.interface = interface

    def evaluate_emittance(self, inputs):
        self.interface.set_parameters(inputs)

        time.sleep(1.0)

        observation = YAGScreen(self.interface,
                                n_samples=5,
                                average_measurements=True,
                                target_charge=4.0e-9,
                               charge_deviation=0.1)

        results = observation.measure_screen()
        logging.debug('done analyzing images')
        
        #remove a bunch of excess info
        pop_args = ['ellipses', 'processed_images', 'raw_images']
        for ele in pop_args:
            results.pop(ele)
        results['total_rms'] = np.sqrt(results['rms_x']**2 + results['rms_y']**2)
        return results


VOCS = {
    'name': 'AWA_Opt',

    'variables': {
        'AWA:Bira3Ctrl:Ch06': [-8, 8],  # DQ7


    },

    'objectives': {
        'total_rms': 'MINIMIZE',
    },

    'constraints': {
        #'rms_x': ['LESS_THAN', 150],
        #'rms_y': ['LESS_THAN', 150],
        #'centroid_offset': ['LESS_THAN', 500],

    },
    'constants': {
    },

}
#x_init = torch.linspace(0.1, 0.7, 10)
#x_initial = torch.cat((x_init, torch.flipud(x_init), x_init)).reshape(-1,1)
#print(x_initial)

x_initial = torch.rand((20, 1))*(0.7-0.1) + 0.1

awa_interface = awa_interface.AWAInterface(use_frame_grabber=False, testing=False)
evaluator = Evaluator(awa_interface)
opt_results = bayesian_optimize(VOCS,
                               evaluator.evaluate_emittance,
                               initial_x = x_initial,
                               n_steps=0,
                               generator_options= {'acquisition_function':acq})


INFO:awa_interface:Starting controller_interface
INFO:awa_interface:Connecting to AWAPGCamera application
INFO:awa_interface:Done
INFO:xopt.bayesian.optimize:started running optimization with generator: <xopt.bayesian.generators.generator.BayesianGenerator object at 0x000002458DC6A340>
INFO:xopt.bayesian.optimize:submitting initial candidates at time 2021-10-19T16:20:48-05:00
INFO:awa_interface:taking n samples 5
INFO:awa_interface:taking n samples 5
INFO:awa_interface:taking n samples 5


INFO:awa_interface:taking n samples 5
INFO:awa_interface:taking n samples 5
INFO:awa_interface:taking n samples 5


INFO:awa_interface:taking n samples 5


INFO:awa_interface:taking n samples 5
INFO:awa_interface:taking n samples 5


INFO:awa_interface:taking n samples 5
INFO:awa_interface:taking n samples 5
INFO:awa_interface:taking n samples 5


INFO:awa_interface:taking n samples 5
INFO:awa_interface:taking n samples 5


INFO:awa_interface:taking n samples 5
INFO:awa_interface:taking n samples 5


INFO:awa_interface:taking n samples 5
INFO:awa_interface:taking n samples 5
INFO:awa_interface:taking n samples 5


INFO:awa_interface:taking n samples 5
INFO:xopt.bayesian.optimize:starting optimization loop


In [3]:
print(opt_results)

{'variables': tensor([[0.4846],
        [0.6051],
        [0.4870],
        [0.2700],
        [0.6253],
        [0.2654],
        [0.4375],
        [0.3958],
        [0.1726],
        [0.1103],
        [0.6320],
        [0.2922],
        [0.3933],
        [0.2831],
        [0.1328],
        [0.1401],
        [0.5791],
        [0.6953],
        [0.5895],
        [0.6529]], dtype=torch.float64), 'objectives': tensor([[118.8024],
        [126.5636],
        [123.0853],
        [123.4973],
        [130.5443],
        [125.6806],
        [120.6654],
        [127.5408],
        [141.3186],
        [138.6027],
        [128.6699],
        [124.4749],
        [117.4239],
        [122.3621],
        [144.7975],
        [136.1983],
        [131.9474],
        [147.6791],
        [136.1401],
        [141.8438]], dtype=torch.float64), 'corrected_objectives': tensor([[-118.8024],
        [-126.5636],
        [-123.0853],
        [-123.4973],
        [-130.5443],
        [-125.6806],
        [-120.66

In [4]:
X = opt_results['variables'][-1]
X_in = dict(zip(VOCS['variables'], X))
res = evaluator.evaluate_emittance(X_in)
print(res['rms_x'])
print(res['rms_y'])
print(res['centroid_offset'])
print(res['n_blobs'])

INFO:awa_interface:taking n samples 5


55.08260737460478
119.54213603871794
41.67735181487354
1.0


In [5]:
print(res.keys())
print(res['charge'])
print(res['rms_y'])

dict_keys(['rms_x', 'rms_y', 'n_blobs', 'image_check', 'centroid_offset', 'total_intensity', 'FWHMX', 'FWHMY', 'FWHML', 'CX', 'CY', 'charge', 'ROI', 'rms_x_std', 'rms_y_std', 'CX_std', 'CY_std', 'n_blobs_std', 'FWHMX_std', 'FWHMY_std', 'centroid_offset_std', 'total_intensity_std', 'total_rms'])
[[-2.78932347e-09  5.91506667e-10 -8.95811272e-10 -1.24895681e-10]
 [-3.93443486e-09  0.00000000e+00 -8.43193281e-10  0.00000000e+00]
 [-3.93443486e-09  0.00000000e+00 -8.43193281e-10  0.00000000e+00]
 [-3.88768350e-09 -8.08733376e-10 -3.11199567e-10  3.69021258e-10]
 [-3.98756000e-09 -1.89247814e-09 -4.10128298e-10  5.29782126e-10]]
119.54213603871794
